External datapoints to leverage:
- Vacations
- Weather
- Lockdowns / curfews
- Fuel prices
- Car traffic

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import arrow
import dateutil.parser
import math

import warnings
warnings.filterwarnings("ignore")

data = pd.read_parquet(Path('data') / 'train.parquet')
test = pd.read_parquet(Path('data') / 'test.parquet')

# converting date into seperate columns for each aspect

data.groupby(['site_name', 'counter_name'])['bike_count'].sum().sort_values(ascending=False).head(30).to_frame()


,,bike_count
site_name,counter_name,
Totem 73 boulevard de Sébastopol,Totem 73 boulevard de Sébastopol S-N,1809231.0
Totem 64 Rue de Rivoli,Totem 64 Rue de Rivoli O-E,1406900.0
Totem 73 boulevard de Sébastopol,Totem 73 boulevard de Sébastopol N-S,1357868.0
67 boulevard Voltaire SE-NO,67 boulevard Voltaire SE-NO,1036575.0
Totem 64 Rue de Rivoli,Totem 64 Rue de Rivoli E-O,914089.0
27 quai de la Tournelle,27 quai de la Tournelle SE-NO,888717.0
Quai d'Orsay,Quai d'Orsay E-O,849724.0
Totem Cours la Reine,Totem Cours la Reine O-E,806149.0
Face au 48 quai de la marne,Face au 48 quai de la marne SO-NE,806071.0


In [2]:
from numpy.lib.shape_base import apply_along_axis
import pandas as pd
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, FunctionTransformer, PolynomialFeatures
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import SplineTransformer
import lightgbm as lgb
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import FeatureUnion
from sklearn.kernel_approximation import Nystroem 

def periodic_spline_transformer(period, n_splines=None, degree=3):
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1  # periodic and include_bias is True
    return SplineTransformer(
        degree=degree,
        n_knots=n_knots,
        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
        extrapolation="periodic",
        include_bias=True,
    )
def na_fill(X):
    if np.isnan(X):
        apply_along_axis
    

def _encode(X):
    #cyclical encoding of dates
    X = X.copy()
    year_norm = 2 * np.pi * X['date'].dt.year / X['date'].dt.year.max()
    month_norm = 2 * np.pi * X['date'].dt.month / X['date'].dt.month.max()
    day_norm = 2 * np.pi * X['date'].dt.day / X['date'].dt.day.max()
    weekday_norm = 2 * np.pi * X['date'].dt.weekday / X['date'].dt.weekday.max()
    hour_norm = 2 * np.pi * X['date'].dt.hour / X['date'].dt.hour.max()
    X.loc[:, 'year_sin'] = np.sin(year_norm)
    X.loc[:, 'year_cos'] = np.cos(year_norm)
    X.loc[:, 'month_sin'] = np.sin(month_norm)
    X.loc[:, 'month_cos'] = np.cos(month_norm)
    X.loc[:, 'day_sin'] = np.sin(day_norm)
    X.loc[:, 'day_cos'] = np.cos(day_norm)
    X.loc[:, 'weekday_sin'] = np.sin(weekday_norm)
    X.loc[:, 'weekday_cos'] = np.cos(weekday_norm)
    X.loc[:, 'hour_sin'] = np.sin(hour_norm)
    X.loc[:, 'hour_cos'] = np.cos(hour_norm)
    #encode dates
    X.loc[:, 'year'] = X['date'].dt.year
    X.loc[:, 'month'] = X['date'].dt.month
    X.loc[:, 'day'] = X['date'].dt.day
    X.loc[:, 'weekday'] = X['date'].dt.weekday
    X.loc[:, 'hour'] = X['date'].dt.hour
    X.loc[:, 'is_weekend'] = np.where(X['weekday'].isin([5,6]), 1,0)
    X.loc[:, 'workday'] = np.where(X['weekday'].isin([0,4]), 1,0)
    return X.drop(columns=["date"]) 
 
def _merge_external_data(X):
    file_path = Path(__file__).parent / 'external_data.csv'
    df_ext = pd.read_csv(file_path, parse_dates=['date'])
    X = X.copy()
    # When using merge_asof left frame need to be sorted
    X['orig_index'] = np.arange(X.shape[0])
    X = pd.merge_asof(X.sort_values('date'), df_ext[['date', 't', 'ff', 'u', 'brent', 'holidays', 'curfew', 'rush hour', 'Taux', 'workplace', 'transit']].sort_values('date'), on='date')
    # Sort back to the original order
    X = X.sort_values('orig_index')
    del X['orig_index']
    return X



def get_estimator():
    #columns
    date_encoder = FunctionTransformer(_encode)
    #cycl_cols = ['month_sin', 'month_cos','day_sin', 'day_cos', 'weekday_sin', 'weekday_cos', 'hour_sin', 'hour_cos']
    date_cols = ['year', 'day']
 
    categorical_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    categorical_cols = ["site_name", "counter_name"]
    binary_cols =  ['curfew', 'rush hour']
    numeric_cols = ['Taux', 't', 'brent', 'ff', 'u', 'transit']
 
    #preprocessor
        
    preprocessor = ColumnTransformer(
        [
            ('date', 'passthrough', date_cols),
            #('cycl', 'passthrough', cycl_cols),
            ("cyclic_month", periodic_spline_transformer(12, n_splines=6), ["month"]),
            ("cyclic_weekday", periodic_spline_transformer(7, n_splines=3), ["weekday"]),
            ("cyclic_hour", periodic_spline_transformer(24, n_splines=12), ["hour"]),
            ('holiday', 'passthrough', binary_cols),  
            ('cat', categorical_encoder, categorical_cols),
            ('numeric', 'passthrough', numeric_cols)
        ]
    )
    hour_workday_interaction = make_pipeline(
        ColumnTransformer(
            [
            ("cyclic_hour", periodic_spline_transformer(24, n_splines=12), ["hour"]),
            ("weekday", FunctionTransformer(lambda x: x == "True"), ["workday"]),
            ]
        ),
        PolynomialFeatures(degree=2, interaction_only=True, include_bias=False),
    )

    regressor = lgb.LGBMRegressor(n_estimators=275, num_leaves=150, importance_type='gain', random_state=0)

    pipe = make_pipeline(
        FunctionTransformer(_merge_external_data, validate=False),
        date_encoder, 
        FeatureUnion(
            [('without', preprocessor),
             ('with', hour_workday_interaction)
             ]
        ),
        regressor)

    return pipe

In [3]:
import problem

X_train, y_train = problem.get_train_data()
X_test, y_test = problem.get_test_data()

In [4]:
data

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,counter_technical_id,latitude,longitude,log_bike_count
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.000000
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.693147
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.000000
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,1.609438
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,2.302585
...,...,...,...,...,...,...,...,...,...,...,...
928450,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,51.0,2021-08-08 18:00:00,2020-11-29,Y2H20114504,48.839770,2.301980,3.951244
928453,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,1.0,2021-08-09 02:00:00,2020-11-29,Y2H20114504,48.839770,2.301980,0.693147
928456,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,61.0,2021-08-09 08:00:00,2020-11-29,Y2H20114504,48.839770,2.301980,4.127134
928459,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,44.0,2021-08-09 10:00:00,2020-11-29,Y2H20114504,48.839770,2.301980,3.806662


In [ ]:
#merging train data
X_train = _merge_external_data(X_train, 'external_data.csv', ['date', 't', 'ff', 'u'])
X_train = _merge_external_data(X_train, 'brent-crude-prices.csv', ['date', 'price'])
X_train = _merge_external_data(X_train, 'school-holidays.csv', ['date', 'vacances_zone_c'])
X_train = _merge_external_data(X_train, 'bank-holidays.csv', ['date', 'val'])

In [ ]:
data

In [ ]:
X_train

In [ ]:
#merging for test data
X_test = _merge_external_data(X_test, 'external_data.csv', ['date', 't', 'ff', 'u'])
X_test = _merge_external_data(X_test, 'brent-crude-prices.csv', ['date', 'price'])
X_test = _merge_external_data(X_test, 'school-holidays.csv', ['date', 'vacances_zone_c'])
X_test = _merge_external_data(X_test, 'bank-holidays.csv', ['date', 'val'])

In [ ]:
car_import = pd.read_csv('./submissions/external_data/car-traffic.csv', sep=';')

In [ ]:
# adding car traffic info to dataframe (in progress still)
car_traffic = car_import[['Libelle', 'date', 'Debit horaire', 'Etat trafic' ]]
car_traffic['Libelle'] = car_traffic['Libelle'].replace(['Bd_Sebastopol', 'Bd_Voltaire', 'Quai_Francois_Mauriac', 'Sevres',
       "Quai_d'Orsay", 'Julia_Bartet', 'Turbigo', 'Quai_de_Grenelle',
       'Quai_Tournelle', 'Concorde', 'Cours_La_Reine']
        , ['Totem 73 boulevard de Sébastopol S-N', '67 boulevard Voltaire SE-NO', '39 quai François Mauriac SE-NO'
        , '90 Rue De Sèvres NE-SO', "Quai d'Orsay E-O", '6 rue Julia Bartet NE-SO'
        , '38 rue Turbigo NE-SO', '36 quai de Grenelle NE-SO', '27 quai de la Tournelle NO-SE'
        , 'Pont de la Concorde S-N','Totem Cours la Reine O-E'])
car_traffic['date'] = pd.to_datetime(car_traffic['date'])

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, FunctionTransformer

date_encoder = FunctionTransformer(_encode_dates)
date_cols = _encode_dates(X_train[['date']]).columns.tolist()



binary_encoding_columns = ['vacances_zone_c', 'val']
one_hot_encoding_columns = ['counter_name', 'site_name']
numeric_cols = ['t', 'ff', 'u', 'price']



preprocessor = ColumnTransformer([
    ('binary-encoder', OrdinalEncoder(), binary_encoding_columns),
    ('one-hot-encoder', OneHotEncoder(handle_unknown='ignore'),
     one_hot_encoding_columns),
    ('numeric', 'passthrough', numeric_cols)
])


In [ ]:
date_cols

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, TimeSeriesSplit

cv = TimeSeriesSplit(n_splits=6)

model = make_pipeline(date_encoder, 
    preprocessor, HistGradientBoostingRegressor(random_state=0)
)

scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='neg_root_mean_squared_error')
print(f'RMSE: {-scores.mean():.3} ± {(-scores).std():.3}')



In [ ]:
date_encoder = FunctionTransformer(_cycl_encode)
date_cols = _cycl_encode(X_train[['date']]).columns.tolist()

categorical_encoder = OneHotEncoder(handle_unknown="ignore")
categorical_cols = ["counter_name", "site_name", 'vacances_zone_c', 'val']

preprocessor = ColumnTransformer([
    ('date', "passthrough", date_cols),
    ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_cols),
    ('numeric', 'passthrough', numeric_cols)
])

regressor = HistGradientBoostingRegressor(random_state=0)

pipe =  make_pipeline(date_encoder, preprocessor, regressor)
pipe.fit(X_train, y_train)

In [ ]:
date_cols

In [ ]:
scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring='neg_root_mean_squared_error', error_score=np.nan)
print(f'RMSE: {-scores.mean():.3} ± {(-scores).std():.3}')


In [ ]:
#more accurate external data csv
import datetime
date_range = pd.DataFrame(pd.date_range(start="2020-09-01 00:00:00",end="2021-10-21 00:00:00", freq='H').to_pydatetime().tolist())

In [ ]:
new_external = _merge_external_data(date_range, 'external_data.csv', ['date', 't', 'ff', 'u', 'brent', 'holidays', 'curfew', 'rush hour', 'Taux', 'bike'])
date_range['date'] = date_range[0]

In [ ]:
new_external.to_csv('new_external.csv')